# 第五章 自动合并检索

在本课程中，我们将探索一种高级的检索聚合（RAG）技术——自动合并检索。这种方法解决了使用传统RAG时遇到的一个关键问题：当检索信息块以填充语言模型（LLM）上下文时，如果这些块过于碎片化，将会影响上下文的连贯性，特别是当这些块的大小较小时。为了克服这一挑战，我们将采用一种自动合并启发式方法，该方法通过将较小的信息块合并到更大的父块中，从而保证了上下文的连贯性。接下来，我们将详细介绍如何实现这一技术，并通过实际案例来演示其应用。

首先，我们需要准备环境和导入必要的库。请注意，为了避免不必要的警告信息干扰我们的操作，我们将使用warnings库来忽略它们。

In [1]:
import warnings
warnings.filterwarnings('ignore')

接下来，我们将导入一些工具函数，设置OpenAI的API密钥，并准备我们的文档。这些文档将作为我们的检索源。

In [2]:
import utils

import os
import openai
# 通过utils获取OpenAI API密钥并设置
openai.api_key = utils.get_openai_api_key()

# 或者这里填入你的OpenAI API密钥
# openai.api_key = "" 

# 或者自定义API密钥和API基础地址，可适用第三方API服务
# openai.api_key = "sk- "  
# openai.api_base = " "

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


我们将使用 `llama_index` 库来读取和处理文档。在这个例子中，我们将加载一个PDF文件，但鼓励大家尝试使用自己的文档。

In [3]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./data/人工智能.pdf"]
).load_data()

为了验证我们的文档已经被正确加载，让我们打印出文档的类型、数量以及第一个文档的内容。

In [4]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

7 

<class 'llama_index.schema.Document'>
Doc ID: a7209ccc-3081-4db0-99c1-ce98c69e21f1
Text: 2/2/24, 2:43 PM ⼈⼯智能  - 维基百科，⾃由的百科全书
https://zh.wikipedia.org/wiki/ ⼈⼯智能 2/13“⼈⼯智能”的各地常⽤名称 中国⼤陆⼈⼯智能 台湾⼈⼯智慧
港澳⼈⼯智能 新⻢⼈⼯智能、⼈⼯智慧 ⽇韩⼈⼯知能 越南智慧⼈造 [展开] [展开] [展开] [展开] [展开] [展开]⼈⼯智能系列内容
主要⽬标 实现⽅式 ⼈⼯智能哲学 历史 技术 术语⼈⼯智能（英语：artiﬁcial intelligence ，缩写为
AI）亦称机器智能，指由⼈制造出来的机器所表现出来的智能。通常⼈⼯
智能是指⽤普通计算机程序来呈现⼈类智能的技术。该词也指出研究这样的智能系统是否能够实现，以及如何实现。同 时，通过 医学 、神经科学
、机器⼈学 及...


## 一、自动合并检索的设置

我们将所有文档合并成一个大的文档对象，以便进行自动合并检索。

In [5]:
from llama_index import Document

# 将所有文档文本合并成一个大文档
document = Document(text="\n\n".join([doc.text for doc in documents]))

In [6]:
# 将中文标点符号替换成英文标点符号，方便后续处理
# 如果是英文文档，可以跳过这一步
# 不处理的话，会导致无法正确切分中文句子，会影响后续sentence_window的大小，导致输入长度大于gpt-3.5-turbo的最大限制
document.text=document.text.replace('。','. ')
document.text=document.text.replace('！','! ')
document.text=document.text.replace('？','? ')

现在，我们将创建一个层次结构节点解析器。这个解析器将帮助我们根据文档内容创建一个层次结构，以便于后续的自动合并操作。

In [7]:
from llama_index.node_parser import HierarchicalNodeParser

# 使用默认设置创建层次结构节点解析器
node_parser = HierarchicalNodeParser.from_defaults(
    chunk_sizes=[2048, 512, 128]
)

通过节点解析器，我们得到了文档中的所有节点。

In [8]:
# 从文档中获取节点
nodes = node_parser.get_nodes_from_documents([document])

为了展示如何具体操作，我们将取出一些叶节点并查看它们的内容。叶节点是层次结构中最小的块，它们将直接参与到后续的合并过程中。

In [9]:
from llama_index.node_parser import get_leaf_nodes

# 获取叶节点并打印其中一个的文本作为示例
leaf_nodes = get_leaf_nodes(nodes)
print(leaf_nodes[30].text)

⼈⼯智能⼯具
与司法部门的⼈类法官相辅相成，提供客观、⼀致的风险评估[13].


我们也可以查看这些叶节点的父节点。父节点包含了它的所有子节点的内容，这对于我们理解自动合并的过程非常重要。

In [10]:
# 根据ID获取节点，并打印一个叶节点的父节点文本作为示例
nodes_by_id = {node.node_id: node for node in nodes}

parent_node = nodes_by_id[leaf_nodes[30].parent_node.node_id]
print(parent_node.text)

⼈⼯智能为地⽅政府服务带来技术突破. ⼈⼯智能代理协助城市规划者基于⽬标导向的蒙特卡罗树搜索进⾏场景规划. ⽬标推理⼈⼯智能代理提供最佳的⼟
地利⽤解决⽅案，帮助我们制定民主的城市⼟地利⽤规划. ⼈⼯智能利⽤在线数据来监控和修改环境威胁政策. 在2019 年⽔危机期间，潜在狄利克雷分配⽅
法确定了Twitter (X) 中讨论最多的主题，这是⼀种朴素的推⽂分类⽅法，对⼲旱的影响和原因、政府响应和潜在解决⽅案等主题进⾏了分类. ⼈⼯智能⼯具
与司法部门的⼈类法官相辅相成，提供客观、⼀致的风险评估[13].


### 1.1、构建索引

我们将构建索引并定义检索器，以便运行查询引擎并评估自动合并检索的效果。

继续构建我们的自动合并检索系统，我们需要使用 OpenAI 的语言模型和一些自定义组件来构建索引，这些组件将支持我们的自动合并逻辑。

首先，我们将初始化一个 OpenAI 的语言模型，这里我们选择使用 GPT-3.5 Turbo ，这是一个强大的模型，适用于各种NLP任务。

In [11]:
from llama_index.llms import OpenAI

# 使用OpenAI的GPT-3.5 Turbo模型
llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

接下来，我们将创建一个服务上下文对象。这个对象将包含我们所需的所有服务，例如嵌入模型和节点解析器，以便我们的系统能够正确地执行自动合并检索。

In [12]:
from llama_index import ServiceContext

# 创建服务上下文，包括LLM模型和节点解析器
auto_merging_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model="local:BAAI/bge-small-zh-v1.5",
    node_parser=node_parser,
)

现在，我们需要将我们的文档节点存储在一个索引中。这个索引将支持快速检索操作，并允许我们根据需要动态合并文档节点。

In [13]:
from llama_index import VectorStoreIndex, StorageContext

# 创建存储上下文并添加文档节点
storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

# 创建自动合并索引
automerging_index = VectorStoreIndex(
    leaf_nodes, storage_context=storage_context, service_context=auto_merging_context
)

# 持久化索引以便未来使用
automerging_index.storage_context.persist(persist_dir="./merging_index")


为了确保我们的系统能够在需要时重新加载索引，我们还提供了一个可选的代码块，用于检查索引文件是否存在，并根据需要加载或重新构建它。

In [14]:
# 可选代码块：检查索引文件是否存在，如果不存在则重建
import os
from llama_index import VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index import load_index_from_storage

if not os.path.exists("./merging_index"):
    storage_context = StorageContext.from_defaults()
    storage_context.docstore.add_documents(nodes)

    automerging_index = VectorStoreIndex(
            leaf_nodes,
            storage_context=storage_context,
            service_context=auto_merging_context
        )

    automerging_index.storage_context.persist(persist_dir="./merging_index")
else:
    automerging_index = load_index_from_storage(
        StorageContext.from_defaults(persist_dir="./merging_index"),
        service_context=auto_merging_context
    )


### 1.2、定义检索器并运行查询引擎

接下来，我们将定义检索器并运行查询引擎。这个过程中，我们将设置一个自动合并检索器，这个检索器将控制文档节点的合并逻辑，以确保检索到的上下文是连贯的。

In [15]:
from llama_index.indices.postprocessor import SentenceTransformerRerank
from llama_index.retrievers import AutoMergingRetriever
from llama_index.query_engine import RetrieverQueryEngine

# 将自动合并索引转换为检索器
automerging_retriever = automerging_index.as_retriever(
    similarity_top_k=12
)

# 定义自动合并检索器
retriever = AutoMergingRetriever(
    automerging_retriever, 
    automerging_index.storage_context, 
    verbose=True
)

# 定义重新排名模块
rerank = SentenceTransformerRerank(top_n=6, model="BAAI/bge-reranker-base")

# 创建自动合并检索查询引擎
auto_merging_engine = RetrieverQueryEngine.from_args(
    automerging_retriever, node_postprocessors=[rerank]
)

为了测试我们的自动合并检索系统，我们将使用一个示例查询：“AI对人类的威胁有哪些？”并展示检索到的响应。

In [16]:
# 使用查询引擎执行查询
auto_merging_response = auto_merging_engine.query(
    "AI对人类的威胁有哪些？"
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


我们可以使用提供的工具函数来美观地展示检索到的响应。

In [17]:
from llama_index.response.notebook_utils import display_response

# 显示查询响应
display_response(auto_merging_response)

**`Final Response:`** AI对人类的威胁主要包括以下几个方面：AI可能会遵循科技发展的加速度理论，具有自我改造创新的能力，进步速度远远超过人类，甚至可能导致人类灭绝的危机。

## 二、整合所有步骤

我们将整合上述所有步骤，并通过 TruLens 对我们的自动合并检索系统进行评估，以验证其性能。

为了将自动合并检索的各个部分整合在一起并进行评估，我们将定义两个高级函数：`build_automerging_index` 和 `get_automerging_query_engine` 。这些函数将帮助我们构建索引并初始化查询引擎，使我们能够以更简洁的方式测试和评估不同的配置。

首先，我们定义 `build_automerging_index` 函数，该函数负责构建自动合并索引。这个函数将接受文档、语言模型、嵌入模型和保存目录作为参数，并构建一个包含所有文档节点的索引。

接下来，我们定义 `get_automerging_query_engine` 函数，该函数负责从自动合并索引中创建查询引擎。它设置了检索器，使之能够根据相似性的 top K 值进行检索，并应用重新排序逻辑来优化检索结果。

In [18]:
import os

from llama_index import (
    ServiceContext,
    StorageContext,
    VectorStoreIndex,
    load_index_from_storage,
)
from llama_index.node_parser import HierarchicalNodeParser
from llama_index.node_parser import get_leaf_nodes
from llama_index import StorageContext, load_index_from_storage
from llama_index.retrievers import AutoMergingRetriever
from llama_index.indices.postprocessor import SentenceTransformerRerank
from llama_index.query_engine import RetrieverQueryEngine


# 定义构建自动合并索引的函数
def build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-zh-v1.5",
    save_dir="merging_index",
    chunk_sizes=None,
):
    chunk_sizes = chunk_sizes or [2048, 512, 128]
    node_parser = HierarchicalNodeParser.from_defaults(chunk_sizes=chunk_sizes)
    nodes = node_parser.get_nodes_from_documents(documents)
    leaf_nodes = get_leaf_nodes(nodes)
    merging_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
    )
    storage_context = StorageContext.from_defaults()
    storage_context.docstore.add_documents(nodes)

    if not os.path.exists(save_dir):
        automerging_index = VectorStoreIndex(
            leaf_nodes, storage_context=storage_context, service_context=merging_context
        )
        automerging_index.storage_context.persist(persist_dir=save_dir)
    else:
        automerging_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=merging_context,
        )
    return automerging_index


# 定义获取自动合并查询引擎的函数
def get_automerging_query_engine(
    automerging_index,
    similarity_top_k=12,
    rerank_top_n=6,
):
    base_retriever = automerging_index.as_retriever(similarity_top_k=similarity_top_k)
    retriever = AutoMergingRetriever(
        base_retriever, automerging_index.storage_context, verbose=True
    )
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )
    auto_merging_engine = RetrieverQueryEngine.from_args(
        retriever, node_postprocessors=[rerank]
    )
    return auto_merging_engine

使用这两个函数，我们可以方便地构建索引和查询引擎，并通过不同的参数配置来测试自动合并检索的效果。下面是如何使用这些函数的示例：

In [19]:
from llama_index.llms import OpenAI

# 构建自动合并索引
index = build_automerging_index(
    [document],
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    save_dir="./merging_index",
)


In [20]:
# 获取自动合并查询引擎
query_engine = get_automerging_query_engine(index, similarity_top_k=6)

## 三、TruLens评估

在构建了自动合并索引和查询引擎后，我们将使用 TruLens 来评估自动合并检索器的性能。 TruLens 是一个用于评估 AI 模型的工具，可以提供详细的性能指标。我们将首先重置 TruLens 数据库，然后进行评估。

In [21]:
from trulens_eval import Tru

# 重置TruLens数据库
Tru().reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


我们将分别评估两层和三层自动合并结构的性能，并使用 TruLens 来记录和展示评估结果。这将帮助我们理解不同层次结构配置对性能的影响，并为我们提供优化自动合并检索策略的洞见。

在进行 TruLens 评估之前，我们需要为两种不同的层次结构设置自动合并索引和查询引擎。这两种结构分别是两层结构和三层结构。通过比较这两种结构的性能，我们可以更好地理解层次结构深度对自动合并检索效果的影响。

### 3.1、两层结构的索引构建

首先，我们将构建一个只有两层的自动合并索引。这种结构简化了创建索引的过程，并减少了检索步骤中所需的工作量。

In [22]:
# 构建具有两层结构的自动合并索引
auto_merging_index_0 = build_automerging_index(
    documents,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    embed_model="local:BAAI/bge-small-zh-v1.5",
    save_dir="merging_index_0",
    chunk_sizes=[2048,512],
)

接着，我们为这个两层结构的自动合并索引设置查询引擎，并保持与之前相同的 top K 和重新排名步骤。

In [23]:
# 获取两层结构的自动合并查询引擎
auto_merging_engine_0 = get_automerging_query_engine(
    auto_merging_index_0,
    similarity_top_k=12,
    rerank_top_n=6,
)

为了进行 TruLens 评估，我们需要定义一个 TruLens 录音器，它将记录查询引擎的响应和评估结果。

In [24]:
from utils import get_prebuilt_trulens_recorder

# 使用TruLens记录器准备评估
tru_recorder = get_prebuilt_trulens_recorder(
    auto_merging_engine_0,
    app_id ='app_0'  # 设置应用ID
)

然后，我们从预先准备好的问题文件中加载评估问题，并定义一个运行评估的函数。这个函数将遍历所有评估问题，并记录每个问题的查询结果。

In [25]:
# 加载评估问题
eval_questions = []
with open('./data/generated_questions.text', 'r') as file:
    for line in file:
        item = line.strip()  # 移除换行符
        eval_questions.append(item)

最后，我们使用上面定义的函数运行评估，并通过 TruLens 获取并展示排行榜。

In [26]:
# 定义运行评估的函数
def run_evals(eval_questions, tru_recorder, query_engine):
    for question in eval_questions:
        with tru_recorder as recording:
            response = query_engine.query(question)
            # 此处运行评估并记录结果

In [27]:
# 对两层结构进行评估
run_evals(eval_questions, tru_recorder, auto_merging_engine_0)

> Merging 3 nodes into parent node.
> Parent node id: 59af9698-d9b1-41fe-ae5d-e83f340390d6.
> Parent node text: 2/2/24, 2:43 PM ⼈⼯智能  - 维基百科，⾃由的百科全书
https://zh.wikipedia.org/wiki/ ⼈⼯智能 3/13
本体论将知识表示为⼀个领域内的⼀
组概...

> Merging 4 nodes into parent node.
> Parent node id: 634498be-6795-49b5-b39d-be498b52c540.
> Parent node text: 2/2/24, 2:43 PM ⼈⼯智能  - 维基百科，⾃由的百科全书
https://zh.wikipedia.org/wiki/ ⼈⼯智能 6/13智能是否可以使⽤⾼级符号表达，如词和想法...

> Merging 3 nodes into parent node.
> Parent node id: 19628c5a-c9a3-479d-a5c0-79b31f364b03.
> Parent node text: 2/2/24, 2:43 PM ⼈⼯智能  - 维基百科，⾃由的百科全书
https://zh.wikipedia.org/wiki/ ⼈⼯智能 4/13
Kismet，⼀个具有表情等社交能⼒
...

> Merging 3 nodes into parent node.
> Parent node id: 2d4c0c46-5bf0-4773-8d94-9ac38ba561a8.
> Parent node text: 2017年6⽉份马云在美国底特律举⾏“链接世界”（Gateway 17）产业⼤会，会上提出⼈⼯智能可能导致第三次世界⼤战，因为前两次产业⾰命都导致两次⼤
战，战争原因并⾮这些创新发明本⾝，⽽是发...

> Merging 2 nodes into parent node.
> Parent node id: eddf7224-f638-4291-bbdd-5b6393ce6852.
> Parent node text: The Behavioral and Brain Science

In [28]:
from trulens_eval import Tru

# 获取并显示排行榜
Tru().get_leaderboard(app_ids=[])

,Context Relevance,Groundedness,Answer Relevance,latency,total_cost
app_id,,,,,
app_0,0.376667,0.9,0.946154,7.866667,0.0


In [29]:
# 运行TruLens仪表板查看详细评估结果
Tru().run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.2.7:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

### 3.2、三层结构的索引构建

接下来，我们将尝试一个更复杂的自动合并索引构建方法，包含三层结构，并可能提高检索的质量。

In [30]:
# 构建具有三层结构的自动合并索引
auto_merging_index_1 = build_automerging_index(
    documents,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    embed_model="local:BAAI/bge-small-zh-v1.5",
    save_dir="merging_index_1",
    chunk_sizes=[2048, 512, 128],  # 三层结构的块大小设置
)

为这个三层结构的自动合并索引设置查询引擎。

In [31]:
# 获取三层结构的自动合并查询引擎
auto_merging_engine_1 = get_automerging_query_engine(
    auto_merging_index_1,
    similarity_top_k=12,
    rerank_top_n=6,
)

同样，我们定义一个 TruLens 记录器，并用相同的评估问题运行评估。

In [32]:
# 为三层结构准备TruLens记录器
tru_recorder = get_prebuilt_trulens_recorder(
    auto_merging_engine_1,
    app_id ='app_1'  # 设置应用ID
)

In [33]:
# 对三层结构进行评估
run_evals(eval_questions, tru_recorder, auto_merging_engine_1)

> Merging 4 nodes into parent node.
> Parent node id: 90b3476a-489a-4fee-beb7-1c5363a50ab7.
> Parent node text: [16]
⼈类解决问题的模式通常是⽤最快捷、直观的判断，⽽不是有意识的、⼀步⼀步的推导，早期⼈⼯智能研究通常使⽤逐步推导的⽅式。[17]⼈⼯智能研究已经
于这种“次表征性的”解决问题⽅法获取进展...

> Merging 2 nodes into parent node.
> Parent node id: 7bab41fa-272e-4847-a568-e22dbc7e9fed.
> Parent node text: 依⽬前的研究⽅向，电脑⽆法突变、苏醒、产⽣⾃我意志，AI也不可能具有创意与智能、同情⼼与审美等这⽅⾯的能⼒。
AI逐渐普及后，将会在企业管理中扮演很重要的⾓⾊，⽽⼈类的管理者应 如何适度的调整⾃...

> Merging 5 nodes into parent node.
> Parent node id: b6df9f33-521b-434b-b26d-8f6876a93184.
> Parent node text: 其它关于 动物 或其它⼈造系统的智能也普遍被认为是⼈⼯智能相关的研究课题。
⼈⼯智能⽬前在 电脑 领域内，得到了愈加⼴泛的发挥。并在 机器⼈ 、经济政治决策、 控制系统 、仿真系统 中得到应⽤。...

> Merging 3 nodes into parent node.
> Parent node id: a7a0bda7-af15-46f6-a27f-28cf2b2357e0.
> Parent node text: 2017年6⽉份马云在美国底特律举⾏“链接世界”（Gateway 17）产业⼤会，会上提出⼈⼯智能可能导致第三次世界⼤战，因为前两次产业⾰命都导致两次⼤
战，战争原因并⾮这些创新发明本⾝，⽽是发...

> Merging 1 nodes into parent node.
> Parent node id: 93ed48c1-a43a-4ebe-ab72-46dd57662601.
> Parent node text: 这与认知科学领域中的表征
感知论点是⼀致的:更⾼的智能需要个体的

In [34]:
from trulens_eval import Tru

# 获取并显示排行榜
Tru().get_leaderboard(app_ids=[])

,Context Relevance,Groundedness,Answer Relevance,latency,total_cost
app_id,,,,,
app_0,0.358788,0.888571,0.9400,7.866667,0.0
app_1,NaN,NaN,0.9125,7.866667,0.0


In [35]:
# 运行TruLens仪表板查看详细评估结果
Tru().run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://192.168.2.7:8501



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

通过比较两层和三层结构的自动合并索引的 TruLens 评估结果，我们可以深入了解不同层次结构对检索性能的影响，并据此优化我们的自动合并策略。